In [10]:
import pandas as pd
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go

athlete_events = pd.read_csv(r"D:/Semester_1/Cloud Technologies/Assignment_2/athlete_events.csv")


Viz_1 = athlete_events[['Team' , 'Medal' , 'Year' , 'NOC']]
Viz_1 = Viz_1.groupby(['Medal', 'Year' , 'Team'],as_index=False).count()
Viz_1 = Viz_1.sort_values(by = 'Year',ascending=True)

filterdViz_1 = ['United States', 'Great Britain' , 'France']
Viz_1 = Viz_1[Viz_1.Team.isin(filterdViz_1)]
Viz_1 = Viz_1.rename(columns={"NOC": "Medal_Count"})

#Dash Implementation

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
server = app.server

year_options = []
for Year in Viz_1['Year'].unique():
    year_options.append({'label':str(Year),'value':Year})
colors = {'Silver': '#D3D3D3','Bronze': '#cd7f32','Gold':'#FFD700'}

app.layout = html.Div([
    html.Div(
                [
                    html.Img(
                        src='https://statathlon.com/wp-content/uploads/2018/01/rio-de-janeiro-2016-summer-olympics-e1467812135773.png',
                        #src = image_filename,
                        className="logo",
                        style={'height':'15%',
                               'width' : '15%'
                              }
                    ),
                    html.H1("Olympics Games - Last 120 years" , style = { 'text-align': 'center'}),                   
                    html.A(
                        html.Button("Learn More", id="learn-more-button"),
                        href="https://plot.ly/dash/pricing/",
                        style = {
                            'float': 'right'
                        }
                    ),
                ],
                className="row",
            ),
    html.Br([]),
    dcc.Tabs([
        dcc.Tab(label='Medal Count', children=[
            
     html.Div([
        dcc.Dropdown(id='year-picker',options=year_options,value=Viz_1['Year'].max())
    ],
    style={'textAlign': 'center','width': '20%', 'display': 'inline-block'}), 
       dcc.Graph(id='graph-with-slider'),
        ]),
        dcc.Tab(label='Tab two', children=[
            dcc.Graph(
                figure={
                    'data': [
                        {'x': [1, 2, 3], 'y': [1, 4, 1],
                            'type': 'bar', 'name': 'SF'},
                        {'x': [1, 2, 3], 'y': [1, 2, 3],
                         'type': 'bar', 'name': u'Montréal'},
                    ]
                }
            )
        ]),
        dcc.Tab(label='Tab three', children=[
            dcc.Graph(
                figure={
                    'data': [
                        {'x': [1, 2, 3], 'y': [2, 4, 3],
                            'type': 'bar', 'name': 'SF'},
                        {'x': [1, 2, 3], 'y': [5, 4, 3],
                         'type': 'bar', 'name': u'Montréal'},
                    ]
                }
            )
        ]),
    ])
])

@app.callback(Output('graph-with-slider', 'figure'),
              [Input('year-picker', 'value')])
def update_figure(selected_year):
    filtered_df = Viz_1[Viz_1['Year'] == selected_year]
    traces = []
    for medal_type in filtered_df['Medal'].unique():
        df_by_medal = filtered_df[filtered_df['Medal'] == medal_type]
        traces.append(go.Bar(
            x=df_by_medal['Team'],
            y=df_by_medal['Medal_Count'],
            name = medal_type,
            marker =dict(color=colors[medal_type])
        ))
        
    return {
        'data': traces,
        'layout': go.Layout(
            title = 'Top 3 Countries: Medal Wise Count for a particular year',
            #barmode = 'stack',
            xaxis={
                'title': 'Country',
                'showgrid' :False,
            },
            yaxis={
                'title': 'Medal Wise Count',
                 'showgrid' :False,
            },
            hovermode='closest'
        )
    }
app.run_server(debug=True, port=8051)


Dash app running on http://127.0.0.1:8051/
